In [486]:
import numpy as np
import snorkel.labeling

In [487]:
randchoices = [*range(0,20,1)]
weights = [0.1, 0.8, 0.8,0.7,3.6,
                                             1.4, 1.7,0.7,59,2.5,
                                             15, 0.2, 0.1, 3.5,0.2,
                                             1.9, 1.5,3.5,0.3,2.49]


In [488]:
random.seed(0)

In [489]:
import random
NUMCLASSES=20


def create_single_labelled_LMatrix(numRows):
    L = np.empty([numRows,NUMCLASSES])
    y = np.empty(numRows)
    for i in range(numRows):
#         y1 = random.randint(0, NUMCLASSES-1)
        r = random.choices(randchoices,weights=weights )
        y1 = r[0]
        L1 = np.empty(NUMCLASSES)
        L1.fill(-1)
        L1[y1] = y1
        
        numlabels = random.randint(0, NUMCLASSES-1)
        if (numlabels < 8):
            for j in range(numlabels):
                y2 = random.randint(0, NUMCLASSES-1)
                L1[y2] = y2
            
        y[i] = y1
        L[i] = L1
    
    return  L, y



In [490]:
L_train1, ytrue = create_single_labelled_LMatrix(50000)

lm = snorkel.labeling.LabelModel(cardinality = 20)
lm.fit(L_train1,  n_epochs=500,lr=0.001, log_freq=100, seed=123)
y_pred = lm.predict(L_train1)

from sklearn.metrics import classification_report
print(classification_report(ytrue, y_pred))


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.09      0.64      0.16        50
         1.0       0.18      0.90      0.30       408
         2.0       0.18      0.87      0.29       403
         3.0       0.19      0.84      0.31       336
         4.0       0.92      0.68      0.78      1798
         5.0       0.33      0.78      0.46       665
         6.0       0.50      0.72      0.59       853
         7.0       0.14      0.91      0.25       324
         8.0       0.96      0.77      0.85     29533
         9.0       0.79      0.68      0.73      1191
        10.0       0.98      0.65      0.78      7552
        11.0       0.14      0.73      0.23       116
        12.0       0.07      0.76      0.13        38
        13.0       0.86      0.65      0.74      1755
        14.0       0.13      0.75      0.22       107
        15.0       0.57      0.74      0.65       911
        16.0       0.46    

/home/srimugunthan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [491]:
len(y_pred[y_pred == -1])

4

In [492]:
import random
NUMCLASSES=20
def create_multi_labelled_LMatrix(numRows):
    L = np.empty([numRows,NUMCLASSES])
    y = np.empty(numRows)
    for i in range(numRows):
        L1 = np.empty(NUMCLASSES)
        L1.fill(-1)
        numlabels = random.randint(0, NUMCLASSES-1)
        for j in range(numlabels):
            y1 = random.randint(0, NUMCLASSES-1)
            L1[y1] = y1
        L[i] = L1
    
    return  L




In [493]:
L_train2 =  create_multi_labelled_LMatrix(15000)




In [494]:
# trial 2

In [495]:
L_train = np.concatenate((L_train1, L_train2))

In [496]:
lm = snorkel.labeling.LabelModel(cardinality = 20)
lm.fit(L_train,  n_epochs=500,lr=0.001, log_freq=100, seed=123)


In [497]:
y_pred_prob = lm.predict_proba(L_train)

y_pred_prob.shape

(65000, 20)

In [498]:
from snorkel.utils import probs_to_preds
probs = y_pred_prob[:50000]
y_pred2 = probs_to_preds(probs)

In [499]:
print(classification_report(ytrue, y_pred2))

              precision    recall  f1-score   support

         0.0       0.02      0.98      0.03        50
         1.0       0.35      0.75      0.48       408
         2.0       0.26      0.72      0.38       403
         3.0       0.19      0.76      0.30       336
         4.0       0.92      0.68      0.78      1798
         5.0       0.44      0.71      0.54       665
         6.0       0.60      0.69      0.64       853
         7.0       0.22      0.73      0.34       324
         8.0       1.00      0.66      0.79     29533
         9.0       0.80      0.68      0.74      1191
        10.0       0.98      0.65      0.78      7552
        11.0       0.05      0.84      0.09       116
        12.0       0.02      0.92      0.03        38
        13.0       0.87      0.65      0.74      1755
        14.0       0.05      0.84      0.09       107
        15.0       0.65      0.70      0.68       911
        16.0       0.58      0.69      0.63       799
        17.0       0.89    

In [500]:
len(y_pred2[y_pred2 == -1])

0

In [501]:
# trial3

In [502]:
import pandas as pd
ltrain2df = pd.DataFrame(L_train2)
ltrain2df["mask"] = False
ltrain2df["truths"] = -2


ltrain1df = pd.DataFrame(L_train1)
ltrain1df["mask"] = True
ltrain1df["truths"] = ytrue
frames = [ltrain1df, ltrain2df]
ltraindf = pd.concat(frames)

In [503]:
ltraindf.shape

(65000, 22)

In [504]:
import sklearn.utils
df = sklearn.utils.shuffle(ltraindf)

# df = ltraindf.sample(frac=1)

In [505]:
numpdf = df.iloc[:, 0:20]

In [506]:
ltrain = numpdf.to_numpy()
mask = df["mask"]
singledf = df[mask]
ytrue = singledf["truths"]

In [507]:
lm = snorkel.labeling.LabelModel(cardinality = 20)
lm.fit(ltrain,  n_epochs=500,lr=0.001, log_freq=100, seed=123)


In [508]:
y_pred_prob = lm.predict_proba(ltrain)

y_pred_prob.shape

(65000, 20)

In [509]:
from snorkel.utils import probs_to_preds
probs = y_pred_prob[mask]
y_pred3 = probs_to_preds(probs)

In [510]:

print(classification_report(ytrue, y_pred3))

              precision    recall  f1-score   support

         0.0       0.02      0.98      0.03        50
         1.0       0.35      0.75      0.48       408
         2.0       0.26      0.72      0.38       403
         3.0       0.19      0.76      0.30       336
         4.0       0.92      0.68      0.78      1798
         5.0       0.44      0.71      0.54       665
         6.0       0.60      0.69      0.64       853
         7.0       0.23      0.73      0.35       324
         8.0       1.00      0.66      0.79     29533
         9.0       0.80      0.68      0.74      1191
        10.0       0.98      0.65      0.78      7552
        11.0       0.05      0.84      0.09       116
        12.0       0.02      0.92      0.03        38
        13.0       0.87      0.65      0.74      1755
        14.0       0.05      0.84      0.09       107
        15.0       0.65      0.70      0.68       911
        16.0       0.58      0.69      0.63       799
        17.0       0.89    

In [511]:
len(y_pred3[y_pred3 == -1])

0